# Word2Vec

This Jupyter notebook is originally from [Kavita Ganesan](https://github.com/kavgan/nlp-text-mining-working-examples/tree/master/word2vec) and was adapted.


# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [2]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


C:\Users\Juergen Brauer\AppData\Local\conda\conda\envs\env_mss\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [3]:
data_file="reviews_data.txt.gz"

nr_reviews_to_show = 3

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        if i==nr_reviews_to_show:
            break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [4]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2019-01-13 19:00:39,220 : INFO : reading file reviews_data.txt.gz...this may take a while
2019-01-13 19:00:39,222 : INFO : read 0 reviews
2019-01-13 19:00:41,367 : INFO : read 10000 reviews
2019-01-13 19:00:43,511 : INFO : read 20000 reviews
2019-01-13 19:00:46,004 : INFO : read 30000 reviews
2019-01-13 19:00:48,348 : INFO : read 40000 reviews
2019-01-13 19:00:50,891 : INFO : read 50000 reviews
2019-01-13 19:00:53,523 : INFO : read 60000 reviews
2019-01-13 19:00:55,617 : INFO : read 70000 reviews
2019-01-13 19:00:57,503 : INFO : read 80000 reviews
2019-01-13 19:00:59,485 : INFO : read 90000 reviews
2019-01-13 19:01:01,423 : INFO : read 100000 reviews
2019-01-13 19:01:03,375 : INFO : read 110000 reviews
2019-01-13 19:01:05,327 : INFO : read 120000 reviews
2019-01-13 19:01:07,310 : INFO : read 130000 reviews
2019-01-13 19:01:09,429 : INFO : read 140000 reviews
2019-01-13 19:01:11,731 : INFO : read 150000 reviews
2019-01-13 19:01:13,713 : INFO : read 160000 reviews
2019-01-13 19:01:15,670

In [12]:
print(type(documents))
print("Nr of reviews=",len(documents))

print(type(documents[0]))
print("Length of first review=",len(documents[0]))

print("First review as list:")
print(documents[0])

<class 'list'>
Nr of reviews= 255404
<class 'list'>
Length of first review= 293
First review as list:
['oct', 'nice', 'trendy', 'hotel', 'location', 'not', 'too', 'bad', 'stayed', 'in', 'this', 'hotel', 'for', 'one', 'night', 'as', 'this', 'is', 'fairly', 'new', 'place', 'some', 'of', 'the', 'taxi', 'drivers', 'did', 'not', 'know', 'where', 'it', 'was', 'and', 'or', 'did', 'not', 'want', 'to', 'drive', 'there', 'once', 'have', 'eventually', 'arrived', 'at', 'the', 'hotel', 'was', 'very', 'pleasantly', 'surprised', 'with', 'the', 'decor', 'of', 'the', 'lobby', 'ground', 'floor', 'area', 'it', 'was', 'very', 'stylish', 'and', 'modern', 'found', 'the', 'reception', 'staff', 'geeting', 'me', 'with', 'aloha', 'bit', 'out', 'of', 'place', 'but', 'guess', 'they', 'are', 'briefed', 'to', 'say', 'that', 'to', 'keep', 'up', 'the', 'coroporate', 'image', 'as', 'have', 'starwood', 'preferred', 'guest', 'member', 'was', 'given', 'small', 'gift', 'upon', 'check', 'in', 'it', 'was', 'only', 'couple',

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [13]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-01-13 19:03:32,845 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-01-13 19:03:32,847 : INFO : collecting all words and their counts
2019-01-13 19:03:32,848 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-13 19:03:33,126 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-01-13 19:03:33,405 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-01-13 19:03:33,723 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-01-13 19:03:34,017 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-01-13 19:03:34,343 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-01-13 19:03:34,655 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76780 word types
2019-01-13 19:03:34,924 : INFO : PROGRESS: a

2019-01-13 19:04:21,857 : INFO : PROGRESS: at 35.65% examples, 1347868 words/s, in_qsize 20, out_qsize 2
2019-01-13 19:04:22,862 : INFO : PROGRESS: at 36.60% examples, 1349421 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:04:23,865 : INFO : PROGRESS: at 37.53% examples, 1350248 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:04:24,883 : INFO : PROGRESS: at 38.48% examples, 1349722 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:04:25,883 : INFO : PROGRESS: at 39.43% examples, 1350815 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:04:26,890 : INFO : PROGRESS: at 40.36% examples, 1351383 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:04:27,898 : INFO : PROGRESS: at 41.27% examples, 1352143 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:04:28,905 : INFO : PROGRESS: at 42.10% examples, 1352937 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:04:29,914 : INFO : PROGRESS: at 42.88% examples, 1353523 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:04:30,919 : INFO : PROGRESS: at 43.54% ex

2019-01-13 19:05:35,219 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-01-13 19:05:35,221 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-01-13 19:05:35,225 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-01-13 19:05:35,229 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-01-13 19:05:35,230 : INFO : training on 207596775 raw words (151750357 effective words) took 113.7s, 1334911 effective words/s
2019-01-13 19:05:35,232 : INFO : training model with 10 workers on 70538 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2019-01-13 19:05:36,256 : INFO : PROGRESS: at 0.42% examples, 1263692 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:05:37,262 : INFO : PROGRESS: at 0.84% examples, 1289570 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:05:38,268 : INFO : PROGRESS: at 1.20% examples, 1299789 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:05:39,275 : INFO : PROGRES

2019-01-13 19:06:49,820 : INFO : PROGRESS: at 30.30% examples, 1233230 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:06:50,839 : INFO : PROGRESS: at 30.72% examples, 1233878 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:06:51,864 : INFO : PROGRESS: at 31.10% examples, 1234673 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:06:52,872 : INFO : PROGRESS: at 31.47% examples, 1235520 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:06:53,890 : INFO : PROGRESS: at 31.84% examples, 1236482 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:06:54,894 : INFO : PROGRESS: at 32.21% examples, 1237341 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:06:55,896 : INFO : PROGRESS: at 32.58% examples, 1238033 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:06:56,899 : INFO : PROGRESS: at 33.05% examples, 1238646 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:06:57,900 : INFO : PROGRESS: at 33.50% examples, 1239284 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:06:58,902 : INFO : PROGRESS: at 33.96% ex

2019-01-13 19:08:09,545 : INFO : PROGRESS: at 61.31% examples, 1207988 words/s, in_qsize 20, out_qsize 0
2019-01-13 19:08:10,554 : INFO : PROGRESS: at 61.60% examples, 1206599 words/s, in_qsize 20, out_qsize 1
2019-01-13 19:08:11,561 : INFO : PROGRESS: at 61.92% examples, 1206409 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:08:12,565 : INFO : PROGRESS: at 62.26% examples, 1206505 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:08:13,586 : INFO : PROGRESS: at 62.63% examples, 1206451 words/s, in_qsize 15, out_qsize 4
2019-01-13 19:08:14,597 : INFO : PROGRESS: at 63.09% examples, 1206731 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:08:15,602 : INFO : PROGRESS: at 63.54% examples, 1207175 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:08:16,619 : INFO : PROGRESS: at 63.96% examples, 1206976 words/s, in_qsize 16, out_qsize 3
2019-01-13 19:08:17,620 : INFO : PROGRESS: at 64.42% examples, 1207305 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:08:18,636 : INFO : PROGRESS: at 64.86% ex

2019-01-13 19:09:29,385 : INFO : PROGRESS: at 90.90% examples, 1178514 words/s, in_qsize 20, out_qsize 2
2019-01-13 19:09:30,399 : INFO : PROGRESS: at 91.13% examples, 1177093 words/s, in_qsize 20, out_qsize 0
2019-01-13 19:09:31,410 : INFO : PROGRESS: at 91.44% examples, 1176747 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:09:32,421 : INFO : PROGRESS: at 91.76% examples, 1176600 words/s, in_qsize 19, out_qsize 0
2019-01-13 19:09:33,440 : INFO : PROGRESS: at 92.07% examples, 1176529 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:09:34,450 : INFO : PROGRESS: at 92.41% examples, 1176453 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:09:35,464 : INFO : PROGRESS: at 92.82% examples, 1176380 words/s, in_qsize 20, out_qsize 0
2019-01-13 19:09:36,472 : INFO : PROGRESS: at 93.21% examples, 1175919 words/s, in_qsize 17, out_qsize 2
2019-01-13 19:09:37,484 : INFO : PROGRESS: at 93.50% examples, 1174530 words/s, in_qsize 18, out_qsize 1
2019-01-13 19:09:38,485 : INFO : PROGRESS: at 93.84% ex

303498017

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [14]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2019-01-13 19:10:38,982 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8666520118713379),
 ('stained', 0.7880571484565735),
 ('unclean', 0.7774798274040222),
 ('grubby', 0.7669922113418579),
 ('dusty', 0.7632491588592529),
 ('smelly', 0.748940110206604),
 ('dingy', 0.7375096678733826),
 ('gross', 0.727153480052948),
 ('disgusting', 0.7210978269577026),
 ('mouldy', 0.7186775803565979)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [15]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9224385619163513),
 ('friendly', 0.8283950090408325),
 ('cordial', 0.8119661808013916),
 ('professional', 0.789492130279541),
 ('attentive', 0.7806856632232666),
 ('curteous', 0.7668532729148865)]

In [16]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6461841464042664),
 ('spain', 0.6417020559310913),
 ('germany', 0.6397117376327515),
 ('lisbon', 0.6203882098197937),
 ('barcelona', 0.6153843402862549),
 ('mexico', 0.610442042350769)]

In [17]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.7834050059318542),
 ('amazed', 0.78287273645401),
 ('astonished', 0.7745355367660522),
 ('dismayed', 0.7672185301780701),
 ('stunned', 0.7611684799194336),
 ('appalled', 0.7562318444252014)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [18]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7317785024642944),
 ('matress', 0.7093439102172852),
 ('mattress', 0.7077168226242065),
 ('blanket', 0.7019851207733154),
 ('quilt', 0.7019296884536743),
 ('pillowcase', 0.6634088754653931),
 ('foam', 0.6375822424888611),
 ('pillows', 0.6329945921897888),
 ('sheets', 0.6327593922615051),
 ('comforter', 0.6164166331291199)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [19]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7489401437293669

In [20]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [21]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.2795017074758513

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [22]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [23]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
